Batch simulation using a list of targets with CT input
---
### Context
This notebook shows an example that takes a list of targets derived fromo EEG locations calculated with SimNIBS charm's processing. 

In this demonstration, BabelBrain's complete pipeline (domain generaiton, transcranial ultrasound modeling and thermal modeling) is executed for 57 targets. 

This example can be modified to any other source of targets.

###  Dataset
We use the dataset corresponding to SDR=0.55 available in the BabelBrain's Zenodo repository. Download it and keep note of the file path to it.

https://doi.org/10.5281/zenodo.7894431

### Create list of EEG locations
In a seperate terminal, we open a conda enviroment pointing to SimNIBS, for example:

`conda activate /Applications/SimNIBS-4.0/simnibs_env/`

Edit the file `eeg_normals.py` to indicate the paths to the SimNIBS output for the dataset, and run the script with

`python eeg_normals.py`

Please note that this step has already being executed. The `eeg_normals.py` script is provided for your reference.


In [ ]:
import numpy as np
import pandas as pd
import sys
import os
import time
from multiprocessing import Process, Queue
import multiprocessing
import nibabel

## Additions to path
We add the paths to BabelBrain

In [ ]:
sys.path.append('/Users/spichardo/Documents/GitHub/BabelBrain')
sys.path.append('/Users/spichardo/Documents/GitHub/BabelBrain/BabelBrain')

In [ ]:
from BabelBrain.CalculateMaskProcess import CalculateMaskProcess
from TranscranialModeling.BabelIntegrationBASE import GetSmallestSOS
#we use the Single Element sim setting
from BabelBrain.CalculateFieldProcess import CalculateFieldProcess 
from BabelBrain.Babel_Thermal.CalculateThermalProcess import CalculateThermalProcess
from ThermalModeling.CalculateTemperatureEffects import GetThermalOutName

## Supportive functions
To read EEG poistions and its normals, and create Brainsight compatible trajectory definitions

In [ ]:
def rotation_matrix_from_vectors(vec1, vec2):
    """ Find the rotation matrix that aligns vec1 to vec2
    :param vec1: A 3d "source" vector
    :param vec2: A 3d "destination" vector
    :return mat: A transform matrix (3x3) which when applied to vec1, aligns it with vec2.
    """
    a, b = (vec1 / np.linalg.norm(vec1)).reshape(3), (vec2 / np.linalg.norm(vec2)).reshape(3)
    v = np.cross(a, b)
    if any(v): #if not all zeros then 
        c = np.dot(a, b)
        s = np.linalg.norm(v)
        kmat = np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
        return np.eye(3) + kmat + kmat.dot(kmat) * ((1 - c) / (s ** 2))

    else:
        return np.eye(3) #cross of all zeros only occurs on identical directions

In [ ]:
templateBSight=\
'''# Version: 13
# Coordinate system: NIfTI:Aligned
# Created by: Brainsight 2.5
# Units: millimetres, degrees, milliseconds, and microvolts
# Encoding: UTF-8
# Notes: Each column is delimited by a tab. Each value within a column is delimited by a semicolon.
# Target Name	Loc. X	Loc. Y	Loc. Z	m0n0	m0n1	m0n2	m1n0	m1n1	m1n2	m2n0	m2n1	m2n2
'''
lineBSight=\
'''{name}\t{X:10.9f}\t{Y:10.9f}\t{Z:10.9f}\t{m0n0:10.9f}\t{m0n1:10.9f}\t{m0n2:10.9f}\t{m1n0:10.9f}\t{m1n1:10.9f}\t{m1n2:10.9f}\t{m2n0:10.9f}\t{m2n1:10.9f}\t{m2n2:10.9f}
'''

**We select those locations not passing by air structures or too close to the ear**

Only Fp1, Fp2, Fpz, LPA, Nz, P9, P10 and RPA are **not** selected.
In total we have 59 out of 67 locations.

In [ ]:
## this file contains YES and NO for each EEG landmark to indicate if we want to include it
SELECTED_FOR_FUS = pd.read_csv('SELECTED_EEG.csv',index_col='Name')['Include'].to_dict()

In [ ]:
def ConvertEEGToBSightFUS(inputcsv='ernie_eeg_normals.csv',
                          outputBSight='ernie_bsight.txt',
                          depthFromSkin=0.0):
    MatResults={}
    A=pd.read_csv(inputcsv)
    with open(outputBSight,'w') as f:
        f.write(templateBSight)
        for index,entry in A.iterrows():
            transform=np.eye(4)
            NormalV=np.array(entry[['Nx','Ny','Nz']].tolist())
            transform[:3,:3]=rotation_matrix_from_vectors(np.array([0,0,-1]),NormalV)
            transform[:3,3]=np.array(entry[['R','A','S']].tolist())+depthFromSkin*NormalV
            outString=lineBSight.format(m0n0=transform[0,0],
                                    m0n1=transform[1,0],
                                    m0n2=transform[2,0],
                                    m1n0=transform[0,1],
                                    m1n1=transform[1,1],
                                    m1n2=transform[2,1],
                                    m2n0=transform[0,2],
                                    m2n1=transform[1,2],
                                    m2n2=transform[2,2],
                                    X=transform[0,3],
                                    Y=transform[1,3],
                                    Z=transform[2,3],
                                    name=entry['Name'])
            if entry['Name'] in SELECTED_FOR_FUS:
                if SELECTED_FOR_FUS[entry['Name']]=='YES':
                    f.write(outString)
                    MatResults[entry['Name']]=transform
    return MatResults

def SaveMatFile(outfile,ID,transform):
    with open(outfile,'w') as f:
        f.write(templateBSight)
        outString=lineBSight.format(m0n0=transform[0,0],
                                m0n1=transform[1,0],
                                m0n2=transform[2,0],
                                m1n0=transform[0,1],
                                m1n1=transform[1,1],
                                m1n2=transform[2,1],
                                m2n0=transform[0,2],
                                m2n1=transform[1,2],
                                m2n2=transform[2,2],
                                X=transform[0,3],
                                Y=transform[1,3],
                                Z=transform[2,3],
                                name=ID)
        f.write(outString)
            

## Function to generate input files from trajectories
It calls the same code as in the BabelBrain's GUI step 1

In [ ]:
def RunMaskGeneration(T1W,
                      ID,
                      simbnibs_path,
                      Mat4Trajectory,
                      ComputingDevice='6800',
                      ComputingBackend=3, 
                      Frequency=500e3,
                      BasePPW=6,
                      TxSystem='Single',
                      bUseCT=True,
                      CT_or_ZTE_input='',
                      ZTERange=[0.1,0.65],
                      HUThreshold=300.0,
                      bReuseFiles=True,
                      CTType=1):
    

    print("*"*40)
    print("*"*5+" Calculating mask.. BE PATIENT... it can take a couple of minutes...")
    print("*"*40)

    deviceName=ComputingDevice
    COMPUTING_BACKEND=ComputingBackend

    T1WIso= T1W
   
    SmallestSoS= GetSmallestSOS(Frequency,bShear=True)

    prefix = ID + '_' + TxSystem +'_%ikHz_%iPPW_' %(int(Frequency/1e3),BasePPW)

    SpatialStep=np.round(SmallestSoS/Frequency/BasePPW*1e3,3) #step of mask to reconstruct , mm
    print("Frequency, SmallestSoS, BasePPW, SpatialStep",Frequency, SmallestSoS, BasePPW,SpatialStep)

    print("Mat4Trajectory",Mat4Trajectory)

    kargs={}
    kargs['SimbNIBSDir']=simbnibs_path
    kargs['SimbNIBSType']='charm'
    kargs['CoregCT_MRI']=True
    kargs['TrajectoryType']='brainsight'
    kargs['Mat4Trajectory']=Mat4Trajectory
    kargs['T1Source_nii']=T1W
    kargs['T1Conformal_nii']=T1WIso
    kargs['nIterationsAlign']=10
    kargs['SpatialStep']=SpatialStep
    kargs['InitialAligment']='HF'
    kargs['Location']=[0,0,0] #This coordinate will be ignored
    kargs['prefix']=prefix
    kargs['bPlot']=False
    kargs['bAlignToSkin']=True
    
    if bUseCT:
        kargs['CT_or_ZTE_input']=CT_or_ZTE_input
        kargs['CTType']=CTType
        if kargs['CTType'] in [2,3]:
            kargs['ZTERange']=ZTERange
        kargs['HUThreshold']=HUThreshold

    # Start mask generation as a separate process.
    queue=Queue()
    print(COMPUTING_BACKEND,deviceName,kargs)
    maskWorkerProcess = Process(target=CalculateMaskProcess, 
                                args=(queue,
                                     COMPUTING_BACKEND,
                                     deviceName),
                                kwargs=kargs)
    maskWorkerProcess.start()      
    # progress.
    T0=time.time()
    bNoError=True
    while maskWorkerProcess.is_alive():
        time.sleep(0.1)
        while queue.empty() == False:
            cMsg=queue.get()
            print(cMsg,end='')
            if '--Babel-Brain-Low-Error' in cMsg:
                bNoError=False

    maskWorkerProcess.join()
    while queue.empty() == False:
        cMsg=queue.get()
        print(cMsg,end='')
        if '--Babel-Brain-Low-Error' in cMsg:
            bNoError=False
    if bNoError:
        TEnd=time.time()
        print('Total time',TEnd-T0)
        print("*"*40)
        print("*"*5+" DONE calculating mask.")
        print("*"*40)
    else:
        print("*"*40)
        print("*"*5+" Error in execution.")
        print("*"*40)
        raise RuntimeError("Error when generating mask!!")

## Function to run transcranial ultrasound modelling
It calls the same code as in the BabelBrain's GUI step 2

In [ ]:
def DistanceOutPlaneToFocus(FocalLength,Diameter):
    return np.sqrt(FocalLength**2-(Diameter/2)**2)
    
def RunAcousticSimSingle(T1W,
                  ID,
                  TxSystem='Single',
                  deviceName='M1',
                  Aperture = 50e-3, #in m
                  FocalLength = 50e-3, #in m
                  Frequency=500e3, #in Hz
                  PPW=6,
                  COMPUTING_BACKEND=3,
                  zLengthBeyonFocalPointWhenNarrow=40e-3, #distance to run simulation beyond target
                  TxAdjustmentZ = 0.0, 
                  bUseCT = True):


    basedir,pID=os.path.split(os.path.split(T1W)[0])
    
    Target=[ID+'_'+TxSystem]

    InputSim=os.path.join(os.path.split(T1W)[0],ID+'_Single_%ikHz_%iPPW_BabelViscoInput.nii.gz' %(int(Frequency/1e3),PPW))
    print('InputSim',InputSim, basedir,pID)

    MaskData = nibabel.load(InputSim)
    FinalMask=np.flip(MaskData.get_fdata(),axis=2)

    VoxelSize=MaskData.header.get_zooms()[0]/1e3
    TargetLocation =np.array(np.where(FinalMask==5.0)).flatten()
    LineOfSight=FinalMask[TargetLocation[0],TargetLocation[1],:]
    StartSkin=np.where(LineOfSight>0)[0].min()
    DistanceFromSkin = (TargetLocation[2]-StartSkin)*VoxelSize
    

    Diameter = Aperture
    DOut=DistanceOutPlaneToFocus(FocalLength,Diameter)
    ZMax=DOut-DistanceFromSkin
    ZMaxSkin = np.round(ZMax*1e3,1)/1e3
    print('ZMaxSkin',ZMaxSkin,'DistanceFromSkin',DistanceFromSkin,'DOut',DOut)
    

    
    #no relocation of Tx
    TxMechanicalAdjustmentX= 0.0 #in m
    TxMechanicalAdjustmentY= 0.0 #in m
    TxMechanicalAdjustmentZ= ZMaxSkin+TxAdjustmentZ  #in m
    print('DistanceFromSkin',DistanceFromSkin,'ZMaxSkin',ZMaxSkin,'TxMechanicalAdjustmentZ',TxMechanicalAdjustmentZ)
    ZIntoSkin =0.0
    CurDistance=ZMaxSkin-TxMechanicalAdjustmentZ
    if CurDistance < 0:
        ZIntoSkin = np.abs(CurDistance)

    extrasuffix='Foc%03.1f_Diam%03.1f_' %(FocalLength*1e3,Aperture*1e3)
   
    Frequencies = [Frequency]
    basePPW=[PPW]
    T0=time.time()
    kargs={}
    kargs['extrasuffix']=extrasuffix
    kargs['ID']=pID
    kargs['deviceName']=deviceName
    kargs['COMPUTING_BACKEND']=COMPUTING_BACKEND
    kargs['basePPW']=basePPW
    kargs['basedir']=basedir+os.sep
    kargs['Aperture']=Aperture
    kargs['FocalLength']=FocalLength
    kargs['TxMechanicalAdjustmentZ']=TxMechanicalAdjustmentZ
    kargs['TxMechanicalAdjustmentX']=TxMechanicalAdjustmentX
    kargs['TxMechanicalAdjustmentY']=TxMechanicalAdjustmentY
    kargs['ZIntoSkin']=ZIntoSkin
    kargs['Frequencies']=Frequencies
    kargs['zLengthBeyonFocalPointWhenNarrow']=zLengthBeyonFocalPointWhenNarrow
    kargs['bUseCT']=bUseCT
    kargs['bPETRA'] = False
    kargs['bMinimalSaving']=False
    kargs['bUseRayleighForWater']= True 

    # Start mask generation as separate process.
    bNoError=True
    queue=Queue()
    T0=time.time()
    fieldWorkerProcess = Process(target=CalculateFieldProcess, 
                                args=(queue,Target,TxSystem),
                                kwargs=kargs)
    fieldWorkerProcess.start()      
    # progress.
    while fieldWorkerProcess.is_alive():
        time.sleep(0.1)
        while queue.empty() == False:
            cMsg=queue.get()
            print(cMsg,end='')
            if '--Babel-Brain-Low-Error' in cMsg:
                bNoError=False  
    fieldWorkerProcess.join()
    while queue.empty() == False:
        cMsg=queue.get()
        print(cMsg,end='')
        if '--Babel-Brain-Low-Error' in cMsg:
            bNoError=False
    if bNoError:
        TEnd=time.time()
        print('Total time',TEnd-T0)
        print("*"*40)
        print("*"*5+" DONE ultrasound simulation.")
        print("*"*40)
    else:
        print("*"*40)
        print("*"*5+" Error in execution.")
        print("*"*40)
        raise RuntimeError("Error when running transcranial simulation!!")

## Function to run thermal modelling
It calls the same code as in the BabelBrain's GUI step 3

In [ ]:
def RunThermalSim(case,
                  AllDC_PRF_Duration,
                  TxSystem='Single',
                  deviceName='M1',
                  COMPUTING_BACKEND=3,
                  BaseIsppa=5.0,
                  bRefocus = False):
    '''
    Define AllDC_PRF_Duration as a list of dictionaries such as
    AllDC_PRF_Duration=[]
    entry={'DC':0.3,'PRF':10.0,'Duration':40.0,'DurationOff':40.0}
    AllDC_PRF_Duration.append(entry)
    '''

    kargs={}
    kargs['deviceName']=deviceName
    kargs['COMPUTING_BACKEND']=COMPUTING_BACKEND
    kargs['Isppa']=BaseIsppa

    kargs['TxSystem']=TxSystem
    if kargs['TxSystem'] in ['CTX_500','Single','H246','BSonix']:
        kargs['sel_p']='p_amp'
    else:
        if bRefocus:
            kargs['sel_p']='p_amp_refocus'
        else:
            kargs['sel_p']='p_amp'


    # Start mask generation as separate process.
    queue=Queue()
    fieldWorkerProcess = Process(target=CalculateThermalProcess, 
                                args=(queue,case,AllDC_PRF_Duration),
                                kwargs=kargs)
    fieldWorkerProcess.start()  

    # Start mask generation as separate process.
    T0=time.time()
    bNoError=True
    
    
    # progress.
    while fieldWorkerProcess.is_alive():
        time.sleep(0.1)
        while queue.empty() == False:
            cMsg=queue.get()
            print(cMsg,end='')
            if '--Babel-Brain-Low-Error' in cMsg:
                bNoError=False  
    fieldWorkerProcess.join()
    while queue.empty() == False:
        cMsg=queue.get()
        print(cMsg,end='')
        if '--Babel-Brain-Low-Error' in cMsg:
            bNoError=False
    if bNoError:
        TEnd=time.time()
        print('Total time',TEnd-T0)
        print("*"*40)
        print("*"*5+" DONE thermal simulation.")
        print("*"*40)
    else:
        print("*"*40)
        print("*"*5+" Error in execution.")
        print("*"*40)
        raise RuntimeError("Error when running thermal simulation!!")

# Run for SDR 0.55

### Convert Ernie EEG locations 
We add **30 mm** inside the brain from the EEG location (skin) as the LIFU target

In [ ]:
MatResults=ConvertEEGToBSightFUS(inputcsv='SDR_0p55_eeg_normals.csv',
                                 outputBSight='SDR0p55_bsight_FUS_30mm.txt',
                                 depthFromSkin=30.0)

In [ ]:
#change path to the right location
BasePath='/Users/spichardo/Documents/TempForSim/SDR_0p55/m2m_SDR_0p55'
T1W = '/Users/spichardo/Documents/TempForSim/SDR_0p55/m2m_SDR_0p55/T1W.nii.gz'
CT = '/Users/spichardo/Documents/TempForSim/SDR_0p55/CT.nii.gz'
bUseCT=True
T1W = os.path.join(BasePath,'T1.nii.gz')
simbnibs_path=BasePath
Frequency=500e3
PPW =6
TxSystem='Single'
Aperture = 50e-3
FocalLength = 50e-3
zLengthBeyonFocalPointWhenNarrow = 40e-3
ComputingDevice='M1' # ID of device

In [ ]:
for ID in MatResults:
    if os.path.isfile(os.path.join(BasePath,ID+'_'+TxSystem+'_%ikHz_%iPPW_BabelViscoInput.nii.gz' %(int(Frequency/1e3),PPW))):
        print('skipping ', ID,', already calculated')
        continue
    Mat4Trajectory=os.path.join(BasePath,ID+'.txt')
    SaveMatFile(Mat4Trajectory,ID,MatResults[ID])
    RunMaskGeneration(T1W,
                      ID,
                      simbnibs_path,
                      Mat4Trajectory,
                      ComputingDevice=ComputingDevice,
                      ComputingBackend=3, #Metal
                      Frequency=Frequency,
                      BasePPW=PPW,
                      TxSystem=TxSystem,
                      CT_or_ZTE_input=CT,
                      bUseCT=bUseCT,
                      CTType=1) # 1 is for real CT, 2 for ZTE and 3 for PETRA
    

## Run all acoustic simulations

In [ ]:
for ID in MatResults:
    if os.path.isfile(os.path.join(BasePath,ID+'_Single_%ikHz_%iPPW_Foc%03.1f_Diam%03.1f_DataForSim.h5' \
                                   %(int(Frequency/1e3),PPW,FocalLength*1e3,Aperture*1e3))):
        print('skipping ', ID,'already calculated')
        continue
    RunAcousticSimSingle(T1W,
                   ID,
                   TxSystem=TxSystem,
                  deviceName=ComputingDevice,
                  Aperture = Aperture, #in m
                  FocalLength = FocalLength, #in m
                  Frequency=Frequency, #in Hz
                  PPW=PPW,
                  COMPUTING_BACKEND=3,
                  zLengthBeyonFocalPointWhenNarrow=zLengthBeyonFocalPointWhenNarrow, #distance to run simulation beyond target
                  bUseCT = bUseCT)
    

## Run all Thermal simulations

In [ ]:
BaseIsppa = 5.0
AllDC_PRF_Duration=[]
entry={'DC':0.3,'PRF':10.0,'Duration':40.0,'DurationOff':40.0}
AllDC_PRF_Duration.append(entry)
AllDC_PRF_Duration


In [ ]:
for ID in MatResults:
    case = os.path.join(BasePath,ID+'_Single_%ikHz_%iPPW_Foc%03.1f_Diam%03.1f_DataForSim.h5' \
                                   %(int(Frequency/1e3),PPW,FocalLength*1e3,Aperture*1e3))
    PrevFiles=[]
    for combination in AllDC_PRF_Duration:
        ThermalName=GetThermalOutName(case,combination['Duration'],
                                                combination['DurationOff'],
                                                combination['DC'],
                                                BaseIsppa,
                                                combination['PRF'])+'.h5'

        if os.path.isfile(ThermalName):
            PrevFiles.append(ThermalName)

    if len(PrevFiles)==len(AllDC_PRF_Duration):
        print('skipping ', ID)
        continue
    RunThermalSim(case,
                  AllDC_PRF_Duration,
                  TxSystem=TxSystem,
                  deviceName=ComputingDevice,
                  COMPUTING_BACKEND=3,
                  BaseIsppa=BaseIsppa,
                  bRefocus = False)
